# Build Full Feature Table

In [1]:
import eegproc as eeg
import pandas as pd

FS = 128
csv_path = "datasets/DREAMER.csv"
dreamer_df = pd.read_csv(csv_path)

features_table: pd.DataFrame = pd.DataFrame()

for patient_id in dreamer_df["patient_index"].unique():
    for video_id in dreamer_df["video_index"].unique():
        mask = (
            (dreamer_df["patient_index"] == patient_id)
            & (dreamer_df["video_index"] == video_id)
        )
        eeg_df = dreamer_df.loc[mask, :].copy()

        if eeg_df.empty:
            continue

        arousal = eeg_df["arousal"].iloc[0]
        valence = eeg_df["valence"].iloc[0]

        eeg_df = eeg_df.drop(columns=["patient_index", "video_index", "arousal", "valence"], errors="ignore")

        clean = eeg.bandpass_filter(
            eeg_df, FS, bands=eeg.FREQUENCY_BANDS, low=0.5, high=45.0, notch_hz=60
        )
        psd = eeg.psd_bandpowers(clean, FS, bands=eeg.FREQUENCY_BANDS)
        hjorth = eeg.hjorth_params(clean, FS)
        shannons = eeg.shannons_entropy(clean, FS, bands=eeg.FREQUENCY_BANDS)
        wt = eeg.wavelet_band_energy(eeg_df, FS, bands=eeg.FREQUENCY_BANDS)
        wt_entropy = eeg.wavelet_entropy(wt, bands=eeg.FREQUENCY_BANDS)

        n = len(psd)
        meta = pd.DataFrame({
            "patient_index": pd.Series([patient_id]*n),
            "video_index": pd.Series([video_id]*n),
            "arousal": pd.Series([int(arousal[1:-1])]*n),
            "valence": pd.Series([int(valence[1:-1])]*n)
        })
        batch = pd.concat([meta, psd, hjorth, shannons, wt, wt_entropy], axis=1)

        features_table = pd.concat([features_table, batch], ignore_index=True)



features_table.to_csv("datasets/features_table.csv")


print(features_table.head())


   patient_index  video_index  arousal  valence     AF3_delta    AF3_theta  \
0              0            0        3        4  17359.703025   781.948892   
1              0            0        3        4  19471.101591   818.849773   
2              0            0        3        4   2089.169556  1084.487456   
3              0            0        3        4   1027.527584  1108.167105   
4              0            0        3        4    425.104735   488.323997   

    AF3_alpha  AF3_betaL    AF3_betaH  AF3_gamma  ...  T7_wentropy  \
0   40.615735  25.348833  1202.992764  11.412149  ...     0.004239   
1   55.737071  38.936099  1299.272677  14.250391  ...     0.003706   
2  129.186026  19.274633   913.468333   8.459519  ...     0.002806   
3   82.756909  32.808039   861.815653   9.261511  ...     0.002853   
4   40.263728  17.024333   828.851980   7.074986  ...     0.002830   

   P7_wentropy  O1_wentropy  O2_wentropy  P8_wentropy  T8_wentropy  \
0     0.000184     0.001556     0.004498

## Add Emipirical Mode Decomposition features

In [ ]:
table: pd.DataFrame = pd.DataFrame()

for patient_id in dreamer_df["patient_index"].unique():
    for video_id in dreamer_df["video_index"].unique():
        mask = (
            (dreamer_df["patient_index"] == patient_id)
            & (dreamer_df["video_index"] == video_id)
        )
        eeg_df = dreamer_df.loc[mask, :].copy()

        if eeg_df.empty:
            continue

        arousal = eeg_df["arousal"].iloc[0]
        valence = eeg_df["valence"].iloc[0]

        eeg_df = eeg_df.drop(columns=["patient_index", "video_index", "arousal", "valence"], errors="ignore")

        imf_energy  = eeg.imf_band_energy(eeg_df, FS).reset_index(drop=True)
        imf_entropy = eeg.imf_entropy(imf_energy).reset_index(drop=True)

        if# check if user video combination is the same in both
        batch = pd.concat([features_table.loc[:len(table) + len(imf_energy) - 1].tail(len(imf_energy) + 1), imf_energy, imf_entropy], axis=1)

        table = pd.concat([table, batch], ignore_index=True)

        break
    break


table.to_csv("datasets/features_table_imf.csv")